In [2]:
import os
import shutil

# Set the TRANSFORMERS_CACHE environment variable to a directory on your D drive
os.environ['TRANSFORMERS_CACHE'] = 'D:\\huggingface_cache'
os.environ['HF_HOME'] = 'D:/huggingface_cache'

# Ensure the cache directory exists
cache_dir = 'D:\\huggingface_cache'
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
    print(f"Created cache directory: {cache_dir}")
else:
    print(f"Cache directory already exists: {cache_dir}")

# Optional: Clear existing cache in the default location
default_cache_dir = os.path.expanduser('~/.cache/huggingface/transformers')
if os.path.exists(default_cache_dir):
    shutil.rmtree(default_cache_dir)
    print(f"Deleted cache directory: {default_cache_dir}")
else:
    print(f"Default cache directory does not exist: {default_cache_dir}")

# Verify environment variable
print(f"TRANSFORMERS_CACHE is set to: {os.getenv('TRANSFORMERS_CACHE')}")


Cache directory already exists: D:\huggingface_cache
Default cache directory does not exist: C:\Users\KARTHIK/.cache/huggingface/transformers
TRANSFORMERS_CACHE is set to: D:\huggingface_cache


In [3]:
%pip install PyMuPDF 
%pip install fitz
%pip install tqdm
%pip install bs4
%pip install PyPDF2
%pip install langchain
%pip install nltk


Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for traits (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [286 lines of output]
      Reading version file C:\Users\KARTHIK\AppData\Local\Temp\pip-install-l19938t3\traits_3aa8aacfb8924fd7967bfa40e7fa1581\traits\version.py
      Package version from version file: ('6.3.2', '6df2ff9bd1d21c74e688aff6f67a19fbbefdd53b')
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\traits
      copying traits\api.py -> build\lib.win-amd64-cpython-311\traits
      copying traits\base_trait_handler.py -> build\lib.win-amd64-cpython-311\traits
      copying traits\constants.py -> build\lib.win-amd64-cpython-311\traits
      copying traits\ctrait.py -> build\lib.win-amd64-cpython-311\traits
      copying traits\editor_factories.py -> build\lib.win-amd64-cpython-311\traits
      copying traits\has_t


  Using cached fitz-0.0.1.dev2-py2.py3-none-any.whl.metadata (816 bytes)
  Using cached configobj-5.0.8-py2.py3-none-any.whl.metadata (3.4 kB)
  Using cached configparser-7.0.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached nibabel-5.2.1-py3-none-any.whl.metadata (8.8 kB)
  Using cached nipype-1.8.6-py3-none-any.whl.metadata (6.6 kB)
  Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached pyxnat-1.6.2-py3-none-any.whl.metadata (5.3 kB)
  Using cached scipy-1.14.0-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached prov-2.0.1-py3-none-any.whl.metadata (3.6 kB)
  Using cached pydot-2.0.0-py3-none-any.whl.metadata (9.6 kB)
  Using cached rdflib-7.0.0-py3-none-any.whl.metadata (11 kB)
  Using cached simplejson-3.19.2-cp311-cp311-win_amd64.whl.metadata (3.2 kB

In [4]:
import warnings

# Ignore all warnings (not recommended unless you know the implications)
warnings.filterwarnings('ignore')

# Filter out specific warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)


In [ ]:
%pip install spacy

In [5]:
import requests
from bs4 import BeautifulSoup
import json
import os
import PyPDF2
import re

# URL of the webpage to scrape
url = "https://www.icar-crida.res.in/publications_annualreports.html"

def fetch_pdf_urls(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        content = BeautifulSoup(response.content, 'html.parser')
        links = content.find_all('a')
        pdf_links = []
        for link in links:
            href = link.get('href')
            if href and '.pdf' in href:
                if not href.startswith('http'):
                    href = 'https://www.icar-crida.res.in/' + href.lstrip('/')
                pdf_links.append(href)
        return pdf_links
    except requests.RequestException as e:
        print(f"Failed to fetch webpage: {url} with error: {e}")
        return []

def download_pdfs(pdf_urls, download_dir='downloaded_pdfs'):
    os.makedirs(download_dir, exist_ok=True)
    downloaded_files = []
    for url in pdf_urls:
        filename = url.split('/')[-1]
        filepath = os.path.join(download_dir, filename)
        if os.path.exists(filepath):
            print(f"Skipping {filename}. Already downloaded.")
            downloaded_files.append(filepath)
            continue
        try:
            response = requests.get(url)
            response.raise_for_status()
            with open(filepath, 'wb') as f:
                f.write(response.content)
            downloaded_files.append(filepath)
            print(f"Downloaded {filename} to {download_dir}")
        except requests.RequestException as e:
            print(f"Failed to download {filename}: {e}")
    return downloaded_files

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    return text.strip()

# def split_text_into_sections(text, section_length=1000):
#     sections = [text[i:i + section_length] for i in range(0, len(text), section_length)]
#     return sections

# def save_sections_to_file(sections, output_file):
#     with open(output_file, 'w', encoding='utf-8') as file:
#         for i, section in enumerate(sections):
#             file.write(f"Section {i+1}:\n")
#             file.write(section + "\n\n")

# Fetch PDF URLs and download PDFs
pdf_urls = fetch_pdf_urls(url)

# Save PDF URLs to a JSON file
with open('icar_crida_report_urls.json', 'w', encoding='utf-8') as f:
    json.dump(pdf_urls, f, ensure_ascii=False, indent=2)

print(pdf_urls)
downloaded_pdfs = download_pdfs(pdf_urls)
print("PDF download and URL scraping completed!")

# Directory to save the text files
text_dir = 'extracted_texts'
os.makedirs(text_dir, exist_ok=True)

# Process each downloaded PDF
for pdf_path in downloaded_pdfs:
    text = extract_text_from_pdf(pdf_path)
    cleaned_text = clean_text(text)
    # sections = split_text_into_sections(cleaned_text)
    txt_filename = os.path.basename(pdf_path).replace('.pdf', '.txt')
    txt_filepath = os.path.join(text_dir, txt_filename)
    # save_sections_to_file(sections, txt_filepath)
    print(f"Extracted, cleaned, and saved text from {pdf_path} to {txt_filepath}")

print("Text extraction, cleaning, splitting, and saving completed!")


['https://www.icar-crida.res.in/./assets_c/img/Annualreports/AR22.pdf', 'https://www.icar-crida.res.in/./assets_c/img/Annualreports/AR21.pdf', 'https://www.icar-crida.res.in/./assets_c/img/Annualreports/AR20.pdf', 'https://www.icar-crida.res.in/./assets/img/Annualreports/AR19.pdf', 'https://www.icar-crida.res.in/./assets_c/img//Annualreports/AR18-19.pdf', 'https://www.icar-crida.res.in/./assets_c/img//Annualreports/AR17-18.pdf', 'https://www.icar-crida.res.in/assets_c/img/Annualreports/AR16-17.pdf', 'https://www.icar-crida.res.in/assets_c/img/Annualreports/AR15-16.pdf', 'https://www.icar-crida.res.in/assets_c/img/Annualreports/AR14-15.pdf', 'https://www.icar-crida.res.in/assets_c/img/Annualreports/AR13-14.pdf', 'https://www.icar-crida.res.in/assets_c/img/Annualreports/AR12-13.pdf', 'https://www.icar-crida.res.in/assets_c/img/Annualreports/AR11-12.pdf', 'https://www.icar-crida.res.in/assets_c/img/Annualreports/AR10-11.pdf', 'https://www.icar-crida.res.in/assets_c/img/Annualreports/AR09-

In [30]:
import fitz
import os
from tqdm import tqdm
from nltk.tokenize import sent_tokenize, word_tokenize

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for pgno in range(len(doc)):
            page = doc.load_page(pgno)
            text += page.get_text().replace('\n', ' ')  # Remove \n characters
        return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return ""

# Function to preprocess text and chunk into sentences
def preprocess_and_chunk_sentences(text, max_chunk_size=300):
    sentences = sent_tokenize(text)  # Tokenize text into sentences
    chunks = []
    current_chunk = []

    for sentence in sentences:
        tokens = word_tokenize(sentence)  # Tokenize each sentence into words
        for token in tokens:
            current_chunk.append(token)
            if len(' '.join(current_chunk)) >= max_chunk_size:
                chunks.append(' '.join(current_chunk))
                current_chunk = []

    # Append any remaining tokens to the last chunk
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Function to process PDFs and directly create text files
def process_pdfs_and_create_files(pdf_dir='downloaded_pdfs', output_folder='processed_pdfs'):
    os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists

    for filename in tqdm(os.listdir(pdf_dir), desc="Processing PDFs"):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, filename)
            text = extract_text_from_pdf(pdf_path)
            if text:
                # Preprocess and chunk text into sentences
                chunks = preprocess_and_chunk_sentences(text)
                # Create text file for each PDF
                output_file_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")
                with open(output_file_path, 'w', encoding='utf-8') as f:
                    for chunk in chunks:
                        f.write(chunk + '\n\n')  # Add new lines between chunks
                print(f"Processed {filename}: {len(chunks)} chunks")

# Process PDFs and directly create text files
process_pdfs_and_create_files()


Processing PDFs:   4%|▎         | 1/28 [00:08<03:51,  8.57s/it]

Processed 2011.pdf: 1064 chunks


Processing PDFs:   7%|▋         | 2/28 [00:31<07:18, 16.88s/it]

Processed 2012-13.pdf: 706 chunks


Processing PDFs:  11%|█         | 3/28 [00:41<05:49, 13.99s/it]

...

Processed FR-2004-07.pdf: 1861 chunks


Processing PDFs:  96%|█████████▋| 27/28 [10:13<00:19, 19.89s/it]

Processed NAIP%20Completion%20Report%20Final.pdf: 737 chunks


Processing PDFs: 100%|██████████| 28/28 [10:28<00:00, 22.46s/it]

Processed PSR%2099-04.pdf: 1981 chunks


Processing PDFs: 100%|██████████| 28/28 [01:53<00:00,  4.06s/it]


File: 2011.pdf, Total Chunks: 1064
File: 2012-13.pdf, Total Chunks: 706
File: 2013-14.pdf, Total Chunks: 569
File: 2014-15.pdf, Total Chunks: 774
File: 2015-16.pdf, Total Chunks: 729
File: AR%202016-17.pdf, Total Chunks: 860
File: AR03-04.pdf, Total Chunks: 1133
File: AR04-05.pdf, Total Chunks: 1206
File: AR06-07.pdf, Total Chunks: 1262
File: AR07-08.pdf, Total Chunks: 1331
File: AR08-09.pdf, Total Chunks: 1463
File: AR09-10.pdf, Total Chunks: 1606
File: AR10-11.pdf, Total Chunks: 1632
File: AR11-12.pdf, Total Chunks: 1914
File: AR12-13.pdf, Total Chunks: 1883
File: AR13-14.pdf, Total Chunks: 1957
File: AR14-15.pdf, Total Chunks: 1672
File: AR15-16.pdf, Total Chunks: 1902
File: AR16-17.pdf, Total Chunks: 2176
File: AR17-18.pdf, Total Chunks: 1802
File: AR18-19.pdf, Total Chunks: 1693
File: AR19.pdf, Total Chunks: 1766
File: AR20.pdf, Total Chunks: 1424
File: AR21.pdf, Total Chunks: 1750
File: AR22.pdf, Total Chunks: 1633
File: FR-2004-07.pdf, Total Chunks: 1861
File: NAIP%20Completion%

# Fixed Size Chunking

In [31]:
# Character-based chunking --- 1m 32.8s
import fitz
import os
from tqdm import tqdm
from nltk.tokenize import sent_tokenize, word_tokenize

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for pgno in range(len(doc)):
            page = doc.load_page(pgno)
            text += page.get_text().replace('\n', ' ')  # Remove \n characters
        return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return ""

# Function to preprocess text and chunk into character-based chunks after NLTK tokenization
def preprocess_and_chunk_characters(text, max_chunk_size=300):
    sentences = sent_tokenize(text)  # Tokenize text into sentences using NLTK
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        tokens = word_tokenize(sentence)  # Tokenize each sentence into words using NLTK
        for token in tokens:
            if len(current_chunk) + len(token) >= max_chunk_size:
                chunks.append(current_chunk)
                current_chunk = ""
            current_chunk += token + " "

    # Append any remaining characters to the last chunk
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks


# Function to process PDFs and directly create text files
def process_pdfs_and_create_files(pdf_dir='downloaded_pdfs', output_folder='character_chunks'):
    os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists

    for filename in tqdm(os.listdir(pdf_dir), desc="Processing PDFs"):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, filename)
            text = extract_text_from_pdf(pdf_path)
            if text:
                # Preprocess and chunk text into characters
                chunks = preprocess_and_chunk_characters(text)
                # Create text file for each PDF
                output_file_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")
                with open(output_file_path, 'w', encoding='utf-8') as f:
                    for chunk in chunks:
                        f.write(chunk + '\n\n')  # Add new lines between chunks
                print(f"Processed {filename}: {len(chunks)} chunks")

# Process PDFs and directly create text files
process_pdfs_and_create_files()


Processing PDFs:   4%|▎         | 1/28 [00:01<00:51,  1.92s/it]

Processed 2011.pdf: 1090 chunks


Processing PDFs:   7%|▋         | 2/28 [00:04<01:02,  2.40s/it]

Processed 2012-13.pdf: 723 chunks


Processing PDFs:  11%|█         | 3/28 [00:06<00:48,  1.96s/it]

...

Processed FR-2004-07.pdf: 1907 chunks


Processing PDFs:  96%|█████████▋| 27/28 [01:28<00:03,  3.23s/it]

Processed NAIP%20Completion%20Report%20Final.pdf: 756 chunks


Processing PDFs: 100%|██████████| 28/28 [01:32<00:00,  3.31s/it]

Processed PSR%2099-04.pdf: 2030 chunks


In [34]:
# Word-based chunking --- 1m 38.7s
import fitz
import os
from tqdm import tqdm
from nltk.tokenize import sent_tokenize, word_tokenize

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for pgno in range(len(doc)):
            page = doc.load_page(pgno)
            text += page.get_text().replace('\n', ' ')  # Remove \n characters
        return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return ""

# Function to preprocess text and chunk into word-based chunks
def word_based_chunking(text, max_words_per_chunk=100):
    words = word_tokenize(text)  # Tokenize text into words using NLTK
    chunks = [' '.join(words[i:i + max_words_per_chunk]) for i in range(0, len(words), max_words_per_chunk)]
    return chunks



# Function to process PDFs and directly create text files
def process_pdfs_and_create_files(pdf_dir='downloaded_pdfs', output_folder='word_chunks'):
    os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists

    for filename in tqdm(os.listdir(pdf_dir), desc="Processing PDFs"):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, filename)
            text = extract_text_from_pdf(pdf_path)
            if text:
                # Preprocess and chunk text into words
                chunks = word_based_chunking(text)
                # Create text file for each PDF
                output_file_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")
                with open(output_file_path, 'w', encoding='utf-8') as f:
                    for chunk in chunks:
                        f.write(chunk + '\n\n')  # Add new lines between chunks
                print(f"Processed {filename}: {len(chunks)} chunks")

# Process PDFs and directly create text files
process_pdfs_and_create_files()


Processing PDFs:   4%|▎         | 1/28 [00:02<01:02,  2.32s/it]

Processed 2011.pdf: 601 chunks


Processing PDFs:   7%|▋         | 2/28 [00:04<01:00,  2.31s/it]

Processed 2012-13.pdf: 397 chunks


Processing PDFs:  11%|█         | 3/28 [00:06<00:47,  1.89s/it]

...

Processed FR-2004-07.pdf: 1012 chunks


Processing PDFs:  96%|█████████▋| 27/28 [01:34<00:03,  3.44s/it]

Processed NAIP%20Completion%20Report%20Final.pdf: 393 chunks


Processing PDFs: 100%|██████████| 28/28 [01:38<00:00,  3.53s/it]

Processed PSR%2099-04.pdf: 1053 chunks


In [35]:
# Sentence-based chunking --- 1m 12.3s
import fitz
import os
from tqdm import tqdm
from nltk.tokenize import sent_tokenize, word_tokenize

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for pgno in range(len(doc)):
            page = doc.load_page(pgno)
            text += page.get_text().replace('\n', ' ')  # Remove \n characters
        return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return ""


# Function to preprocess text and chunk into sentence-based chunks
def sentence_based_chunking(text, max_sentences_per_chunk=5):
    sentences = sent_tokenize(text)  # Tokenize text into sentences using NLTK
    chunks = [' '.join(sentences[i:i + max_sentences_per_chunk]) for i in range(0, len(sentences), max_sentences_per_chunk)]
    return chunks


# Function to process PDFs and directly create text files
def process_pdfs_and_create_files(pdf_dir='downloaded_pdfs', output_folder='sentence_chunks'):
    os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists

    for filename in tqdm(os.listdir(pdf_dir), desc="Processing PDFs"):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, filename)
            text = extract_text_from_pdf(pdf_path)
            if text:
                # Preprocess and chunk text into words
                chunks = sentence_based_chunking(text)
                # Create text file for each PDF
                output_file_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")
                with open(output_file_path, 'w', encoding='utf-8') as f:
                    for chunk in chunks:
                        f.write(chunk + '\n\n')  # Add new lines between chunks
                print(f"Processed {filename}: {len(chunks)} chunks")

# Process PDFs and directly create text files
process_pdfs_and_create_files()


Processing PDFs:   4%|▎         | 1/28 [00:01<00:31,  1.17s/it]

Processed 2011.pdf: 514 chunks


Processing PDFs:   7%|▋         | 2/28 [00:02<00:34,  1.32s/it]

Processed 2012-13.pdf: 387 chunks


Processing PDFs:  11%|█         | 3/28 [00:03<00:28,  1.14s/it]

...

Processed FR-2004-07.pdf: 738 chunks


Processing PDFs:  96%|█████████▋| 27/28 [01:09<00:02,  2.56s/it]

Processed NAIP%20Completion%20Report%20Final.pdf: 511 chunks


Processing PDFs: 100%|██████████| 28/28 [01:12<00:00,  2.58s/it]

Processed PSR%2099-04.pdf: 955 chunks


In [36]:
# Slide Window chunking --- 1m 15.6s
import fitz
import os
from tqdm import tqdm
from nltk.tokenize import sent_tokenize, word_tokenize

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for pgno in range(len(doc)):
            page = doc.load_page(pgno)
            text += page.get_text().replace('\n', ' ')  # Remove \n characters
        return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return ""



# Function to preprocess text and perform slide window chunking based on sentences
def slide_window_chunking(text, window_size=5):
    sentences = sent_tokenize(text)  # Tokenize text into sentences using NLTK
    chunks = []
    for i in range(len(sentences) - window_size + 1):
        chunk = ' '.join(sentences[i:i + window_size])
        chunks.append(chunk)
    return chunks


# Function to process PDFs and directly create text files
def process_pdfs_and_create_files(pdf_dir='downloaded_pdfs', output_folder='window_chunks'):
    os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists

    for filename in tqdm(os.listdir(pdf_dir), desc="Processing PDFs"):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, filename)
            text = extract_text_from_pdf(pdf_path)
            if text:
                # Preprocess and chunk text into words
                chunks = slide_window_chunking(text)
                # Create text file for each PDF
                output_file_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")
                with open(output_file_path, 'w', encoding='utf-8') as f:
                    for chunk in chunks:
                        f.write(chunk + '\n\n')  # Add new lines between chunks
                print(f"Processed {filename}: {len(chunks)} chunks")

# Process PDFs and directly create text files
process_pdfs_and_create_files()


Processing PDFs:   4%|▎         | 1/28 [00:01<00:40,  1.52s/it]

Processed 2011.pdf: 2565 chunks


Processing PDFs:   7%|▋         | 2/28 [00:02<00:37,  1.44s/it]

Processed 2012-13.pdf: 1929 chunks


Processing PDFs:  11%|█         | 3/28 [00:04<00:32,  1.28s/it]

...

Processed FR-2004-07.pdf: 3683 chunks


Processing PDFs:  96%|█████████▋| 27/28 [01:11<00:02,  2.72s/it]

Processed NAIP%20Completion%20Report%20Final.pdf: 2549 chunks


Processing PDFs: 100%|██████████| 28/28 [01:15<00:00,  2.70s/it]

Processed PSR%2099-04.pdf: 4769 chunks


In [38]:
# Sliding Window chunking --- 1m 45.0s
import fitz
import os
from tqdm import tqdm
from nltk.tokenize import sent_tokenize, word_tokenize

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for pgno in range(len(doc)):
            page = doc.load_page(pgno)
            text += page.get_text().replace('\n', ' ')  # Remove \n characters
        return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return ""



# Function to perform slide window chunking based on words
def slide_window_chunking(text, window_size=100, overlap=50):
    words = word_tokenize(text)  # Tokenize text into words using NLTK
    chunks = []
    for i in range(0, len(words) - window_size + 1, overlap):
        chunk = ' '.join(words[i:i + window_size])
        chunks.append(chunk)
    return chunks


# Function to process PDFs and directly create text files
def process_pdfs_and_create_files(pdf_dir='downloaded_pdfs', output_folder='sliding_window_chunks'):
    os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists

    for filename in tqdm(os.listdir(pdf_dir), desc="Processing PDFs"):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, filename)
            text = extract_text_from_pdf(pdf_path)
            if text:
                # Preprocess and chunk text into words
                chunks = slide_window_chunking(text)
                # Create text file for each PDF
                output_file_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")
                with open(output_file_path, 'w', encoding='utf-8') as f:
                    for chunk in chunks:
                        f.write(chunk + '\n\n')  # Add new lines between chunks
                print(f"Processed {filename}: {len(chunks)} chunks")

# Process PDFs and directly create text files
process_pdfs_and_create_files()


Processing PDFs:   0%|          | 0/28 [00:00<?, ?it/s]

Processing PDFs:   4%|▎         | 1/28 [00:02<00:55,  2.07s/it]

Processed 2011.pdf: 1199 chunks


Processing PDFs:   7%|▋         | 2/28 [00:04<01:02,  2.39s/it]

Processed 2012-13.pdf: 791 chunks


...

Processed FR-2004-07.pdf: 2021 chunks


Processing PDFs:  96%|█████████▋| 27/28 [01:38<00:03,  3.44s/it]

Processed NAIP%20Completion%20Report%20Final.pdf: 784 chunks


Processing PDFs: 100%|██████████| 28/28 [01:44<00:00,  3.75s/it]

Processed PSR%2099-04.pdf: 2103 chunks


In [39]:
# Context based chunking --- 1m 4.0s
import fitz
import os
from tqdm import tqdm
from nltk.tokenize import sent_tokenize, word_tokenize

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for pgno in range(len(doc)):
            page = doc.load_page(pgno)
            text += page.get_text().replace('\n', ' ')  # Remove \n characters
        return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return ""



# Function to perform context-based chunking based on sentences
def context_based_chunking(text, max_words_per_chunk=200):
    sentences = sent_tokenize(text)  # Tokenize text into sentences using NLTK
    chunks = []
    current_chunk = []

    for sentence in sentences:
        words = sentence.split()  # Split sentence into words
        if len(' '.join(current_chunk)) + len(words) <= max_words_per_chunk:
            current_chunk.extend(words)
        else:
            if current_chunk:
                chunks.append(' '.join(current_chunk))
                current_chunk = words

    # Append any remaining words to the last chunk
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks


# Function to process PDFs and directly create text files
def process_pdfs_and_create_files(pdf_dir='downloaded_pdfs', output_folder='context_chunks'):
    os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists

    for filename in tqdm(os.listdir(pdf_dir), desc="Processing PDFs"):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, filename)
            text = extract_text_from_pdf(pdf_path)
            if text:
                # Preprocess and chunk text into words
                chunks = context_based_chunking(text)
                # Create text file for each PDF
                output_file_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")
                with open(output_file_path, 'w', encoding='utf-8') as f:
                    for chunk in chunks:
                        f.write(chunk + '\n\n')  # Add new lines between chunks
                print(f"Processed {filename}: {len(chunks)} chunks")

# Process PDFs and directly create text files
process_pdfs_and_create_files()


Processing PDFs:   4%|▎         | 1/28 [00:01<00:31,  1.17s/it]

Processed 2011.pdf: 971 chunks


Processing PDFs:   7%|▋         | 2/28 [00:03<00:47,  1.82s/it]

Processed 2012-13.pdf: 708 chunks


Processing PDFs:  11%|█         | 3/28 [00:04<00:35,  1.44s/it]

...

Processed FR-2004-07.pdf: 1812 chunks


Processing PDFs:  96%|█████████▋| 27/28 [01:00<00:02,  2.18s/it]

Processed NAIP%20Completion%20Report%20Final.pdf: 689 chunks


Processing PDFs: 100%|██████████| 28/28 [01:03<00:00,  2.29s/it]

Processed PSR%2099-04.pdf: 1994 chunks


In [ ]:
# import fitz
# import os
# import json
# from tqdm import tqdm
# from nltk.tokenize import sent_tokenize
# from langchain_text_splitters import NLTKTextSplitter
# from nltk.tokenize import sent_tokenize, word_tokenize

# # Function to extract text from PDF
# def extract_text_from_pdf(pdf_path):
#     try:
#         doc = fitz.open(pdf_path)
#         text = ""
#         for pgno in range(len(doc)):
#             page = doc.load_page(pgno)
#             text += page.get_text().replace('\n', ' ')  # Remove \n characters
#         return text
#     except Exception as e:
#         print(f"Error reading {pdf_path}: {e}")
#         return ""

# # Function to preprocess text and chunk into sentences
# def preprocess_and_chunk_sentences(text, max_chunk_size=300):
#     sentences = sent_tokenize(text)  # Tokenize text into sentences
#     chunks = []
#     current_chunk = []

#     for sentence in sentences:
#         tokens = word_tokenize(sentence)  # Tokenize each sentence into words
#         for token in tokens:
#             current_chunk.append(token)
#             if len(' '.join(current_chunk)) >= max_chunk_size:
#                 chunks.append(' '.join(current_chunk))
#                 current_chunk = []

#     # Append any remaining tokens to the last chunk
#     if current_chunk:
#         chunks.append(' '.join(current_chunk))

#     return chunks


# # Function to process PDFs and preprocess text
# def process_pdfs(pdf_dir='downloaded_pdfs'):
#     pdf_texts = []
#     for filename in tqdm(os.listdir(pdf_dir), desc="Processing PDFs"):
#         if filename.endswith('.pdf'):
#             pdf_path = os.path.join(pdf_dir, filename)
#             text = extract_text_from_pdf(pdf_path)
#             if text:
#                 # Preprocess and chunk text into sentences
#                 chunks = preprocess_and_chunk_sentences(text)
#                 pdf_texts.append({
#                     'filename': filename,
#                     'chunks': chunks,
#                     'total_chunks': len(chunks)
#                 })
#     return pdf_texts

# # Process PDFs and preprocess text
# pdf_texts = process_pdfs()

# # Print number of chunks for each PDF
# for pdf in pdf_texts:
#     print(f"File: {pdf['filename']}, Total Chunks: {pdf['total_chunks']}")

# # Dump pdf_texts to JSON file
# output_file = 'icar_crida_reports_chunking_pdfs'
# with open(output_file, 'w', encoding='utf-8') as f:
#     json.dump(pdf_texts, f, ensure_ascii=False, indent=2)

# print(f"PDF processing completed. Results saved to {output_file}")


In [26]:

# num_items = get_json_length(output_file)
# print(f"Number of items in '{json_file}': {num_items}")


Number of items in 'icar_crida_reports_processed_pdfs.json': 28


In [29]:
# import json
# import os

# # Function to read JSON file and store each item's chunks in separate text files
# def store_chunks_in_text_files(json_file, output_folder):
#     try:
#         # Ensure the output folder exists; create it if it doesn't
#         os.makedirs(output_folder, exist_ok=True)

#         with open(json_file, 'r', encoding='utf-8') as f:
#             data = json.load(f)

#             for item in data:
#                 # Extract filename from item (assuming 'filename' is a key in your JSON)
#                 filename = item.get('filename')
#                 if filename:
#                     # Construct file path with .txt extension
#                     file_path = os.path.join(output_folder, filename + '.txt')

#                     # Extract chunks to write to text file
#                     chunks = item.get('chunks', [])

#                     # Write chunks to text file
#                     with open(file_path, 'w', encoding='utf-8') as out_f:
#                         for chunk in chunks:
#                             out_f.write(chunk + '\n\n')  # Add new lines between chunks

#         print(f"Stored chunks from '{json_file}' in '{output_folder}' as text files")
#     except Exception as e:
#         print(f"Error storing chunks: {e}")

# # Example usage:
# json_file = 'icar_crida_reports_processed_pdfs.json'
# output_folder = 'icar_crida_reports_chunking_pdfs'

# store_chunks_in_text_files(json_file, output_folder)


Stored chunks from 'icar_crida_reports_processed_pdfs.json' in 'icar_crida_reports_chunking_pdfs' as text files


In [7]:

# # # Fixed-size chunking
# # from langchain.text_splitter import CharacterTextSplitter
# # text_splitter = CharacterTextSplitter(
# #     separator = "\n\n",
# #     chunk_size = 256,
# #     chunk_overlap  = 20
# # )
# # docs = text_splitter.create_documents([text])
# # for doc in docs:
# #     print(len(doc.page_content))
# # docs
# # Import the necessary module from langchain
# from langchain.text_splitter import CharacterTextSplitter

# # Print the original text length for verification
# print("Original Text Length:", len(text))
# print("=" * 40)

# # Initialize the CharacterTextSplitter with the correct parameters
# text_splitter = CharacterTextSplitter(
#     chunk_size=100,     # Desired chunk size
#     chunk_overlap=20    # Overlap between chunks
# )

# # Create documents (chunks) from the text
# docs = text_splitter.create_documents([text])

# # Print intermediate results for debugging
# print("Chunks Created:", len(docs))
# if docs:
#     for i, doc in enumerate(docs):
#         print(f"Chunk {i+1} Length: {len(doc.page_content)}")
#         print(doc.page_content)
#         print('-' * 40)  # Separator for readability
# else:
#     print("No chunks were created. Please check the text and parameters.")


In [8]:
# docs = text.split(".")
# # docs
# # not effective at all

In [9]:
# from langchain.text_splitter import NLTKTextSplitter
# text_splitter = NLTKTextSplitter(chunk_size = 300, chunk_overlap = 20)
# docs = text_splitter.split_text(text)
# print(len(docs))
# print(len(docs[0]))
# docs

In [10]:
# !python -m spacy download en_core_web_sm

In [11]:
# from langchain.text_splitter import SpacyTextSplitter
# text_splitter = SpacyTextSplitter()
# docs = text_splitter.split_text(text)
# docs

In [12]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(
#     # Set a really small chunk size, just to show.
#     chunk_size = 256,
#     chunk_overlap  = 20
# )

# docs = text_splitter.create_documents([text])
# docs

In [13]:
# from langchain.text_splitter import MarkdownTextSplitter

# markdown_splitter = MarkdownTextSplitter(chunk_size=100, chunk_overlap=0)
# docs = markdown_splitter.create_documents([text])
# docs

In [14]:
# from langchain.text_splitter import LatexTextSplitter
# latex_splitter = LatexTextSplitter(chunk_size=100, chunk_overlap=0)
# docs = latex_splitter.create_documents([text])
# print(len(docs))
# docs

In [15]:
# %pip install --quiet langchain_experimental langchain_openai